In [53]:
import os
import cv2
from mtcnn.mtcnn import MTCNN
from glob import iglob
import shutil

enroll_folder = 'person'    
final_folder = 'person_processed'

ffolders = [name for name in os.listdir(final_folder)]
efolders = [name for name in os.listdir(enroll_folder) if name not in ffolders]

def expand_bb(bb, shp, percentage=0.25):
    
    wpadding = int(bb[3] * percentage) # 25% increase
    hpadding = int(bb[2] * percentage)

    det = [0,0,0,0]
    det[0] = max(bb[0] - wpadding, 0)
    det[1] = max(bb[1] - hpadding,0)
    det[2] = min(bb[2] + bb[0] + wpadding,shp[1])
    det[3] = min(bb[3] + bb[1] + hpadding,shp[0])
    
    return det

In [54]:
if efolders:
    detector = MTCNN()
    #print(efolders)
    for f in efolders:
        for filename in iglob(os.path.join(enroll_folder, f,'*.jpg'),recursive=False):
            #print(filename)
            im = cv2.imread(filename)
            box = detector.detect_faces(im)
            box = [i for i in box if i['confidence'] >= 0.9 ]
            bb = box[0]['box']
            det = expand_bb(bb, [9999,9999], 0)
            cv2.imwrite(filename + '_mtcnn',im[det[1]:det[3], det[0]:det[2]].copy())
            

    for f in efolders:
        #shutil.copytree(os.path.join(os.getcwd(),enroll_folder,f), os.path.join(os.getcwd(),final_folder,f))
        shutil.move(os.path.join(enroll_folder,f), os.path.join(final_folder,f))
    #print(ffolders)
    #from mtcnn.mtcnn import MTCNN
    #detector = MTCNN()

    print('Enroll Complete. Please re-train..')
else:
    print('No new enrollment found..')

Enroll Complete. Please re-train..
